In [2]:
#Read in Homicide csv

In [3]:
%matplotlib inline
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import gmaps
import requests
import json

# Google developer API key
from config import gkey

# Access maps with unique API key
#gmaps.configure(api_key=gkey)

In [4]:
homicide_path= "Homicide.csv"

In [5]:
homicide_rawdata = pd.read_csv(homicide_path)

In [6]:
homicide_rawdata.head()

,X,Y,Index_,Event_Unique_ID,Occurrence_year,Division,Homicide_Type,Occurrence_Date,Hood_ID,Neighbourhood,Lat,Long,ObjectId
0,-79.392853,43.685028,1,111878,2004,D53,Other,2004-01-03T05:00:00.000Z,98,Rosedale-Moore Park (98),43.685028,-79.392853,1
1,-79.234962,43.781536,2,125755,2004,D42,Shooting,2004-01-08T05:00:00.000Z,137,Woburn (137),43.781536,-79.234962,2
2,-79.206894,43.810860,3,136086,2004,D42,Shooting,2004-01-08T05:00:00.000Z,132,Malvern (132),43.810860,-79.206894,3
3,-79.434403,43.670475,4,148623,2004,D13,Shooting,2004-01-25T05:00:00.000Z,93,Dovercourt-Wallace Emerson-Junction (93),43.670475,-79.434403,4
4,-79.203865,43.823543,5,148619,2004,D42,Shooting,2004-01-25T05:00:00.000Z,131,Rouge (131),43.823543,-79.203865,5


In [8]:
#select relevant columns
homicide_data = homicide_rawdata.loc[:, ['Index_','Occurrence_year',"Lat","Long",'Neighbourhood']]
#rename column names 
homicide_data .rename(columns={'Index_':'Homicide_Index',
                          'Occurrence_year':'Homicide_Year',
                          'Lat':'Homicide_Lat',
                          'Long':'Homicide_Lng',
                          'Neighbourhood':'Homicide_Neighbourhood'},inplace=True)
 
#select only 2015 records
homicide_data = homicide_data.loc[homicide_data["Homicide_Year"]==2016]

homicide_data.head()


,Homicide_Index,Homicide_Year,Homicide_Lat,Homicide_Lng,Homicide_Neighbourhood
658,1008,2016,43.667374,-79.381638,Church-Yonge Corridor (75)
662,1009,2016,43.705143,-79.529404,Weston (113)
666,1010,2016,43.742729,-79.540337,Humbermede (22)
670,1011,2016,43.704571,-79.396797,Mount Pleasant West (104)
674,1012,2016,43.657211,-79.404076,University (79)


In [9]:
#check if any missing values and found none
homicide_data.count()

Homicide_Index            75
Homicide_Year             75
Homicide_Lat              75
Homicide_Lng              75
Homicide_Neighbourhood    75
dtype: int64

In [10]:
locations_homicie = homicide_data[["Homicide_Lat", "Homicide_Lng"]]
print(locations_homicie)

     Homicide_Lat  Homicide_Lng
658     43.667374    -79.381638
662     43.705143    -79.529404
666     43.742729    -79.540337
670     43.704571    -79.396797
674     43.657211    -79.404076
678     43.657211    -79.404076
682     43.649132    -79.437943
685     43.708019    -79.341385
780     43.737808    -79.520134
783     43.650467    -79.371910
785     43.752975    -79.258072
788     43.669415    -79.382492
791     43.677151    -79.334877
794     43.657307    -79.373459
797     43.666161    -79.352913
800     43.641914    -79.412705
803     43.739586    -79.222824
806     43.739586    -79.222824
809     43.739586    -79.222824
812     43.660065    -79.352310
867     43.637348    -79.425339
868     43.768520    -79.500557
869     43.774399    -79.253548
870     43.670212    -79.457031
871     43.734100    -79.609917
873     43.700603    -79.517792
875     43.747040    -79.600159
877     43.699532    -79.556374
879     43.764053    -79.205627
881     43.655464    -79.412613
..      

In [11]:
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [13]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations_homicie)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

In [14]:
# Loop through the Homocide_pd and run a lat/long search for each city
for index, row in homicide_data.iterrows():
    

    latitude = row['Homicide_Lat']
    longitude = row['Homicide_Lng']
#Build the query url
    base_url = "https://maps.googleapis.com/maps/api/geocode/json?key=" + gkey + "&latlng=" + f"{latitude},{longitude}"



    # make request
    postalcode_neigh = requests.get(base_url)
    
    
    # convert to json
    postalcode_neigh = postalcode_neigh.json()

    homicide_data.loc[index, "Homicide_Postal Code"] = postalcode_neigh["results"][1]["address_components"][-1]["long_name"]
    homicide_data.loc[index, "Homicide_Neighborhood"] = postalcode_neigh["results"][0]["address_components"][1]["long_name"]

# Visualize to confirm lat lng appear
homicide_data.head()


,Homicide_Index,Homicide_Year,Homicide_Lat,Homicide_Lng,Homicide_Neighbourhood,Homicide_Postal Code,Homicide_Neighborhood
658,1008,2016,43.667374,-79.381638,Church-Yonge Corridor (75),M4Y 2E5,Gloucester Street
662,1009,2016,43.705143,-79.529404,Weston (113),M9N 1Z5,Weston Road
666,1010,2016,43.742729,-79.540337,Humbermede (22),M9M 2V4,North York
670,1011,2016,43.704571,-79.396797,Mount Pleasant West (104),M4S 1V6,Soudan Avenue
674,1012,2016,43.657211,-79.404076,University (79),M5T 1S6,Brunswick Avenue
